In [1]:
# Импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")

%matplotlib inline

SEED = 42

# from sklearn import metrics #метрики
# from sklearn import model_selection #методы разделения и валидации
# from sklearn import linear_model #линейные модели
# from sklearn import tree #деревья решений

# from scipy import stats
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn.metrics import mean_squared_error, r2_score, make_scorer

# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import cross_validate
# from sklearn.pipeline import Pipeline

import sympy

from sympy import (
    Symbol, 
    S,
    simplify, 
    latex,
    sin,
    cos,
    exp,
    log,
    sqrt,
    FiniteSet,
    Union,
    Interval,
    Contains,
    ConditionSet,
    Eq,
    solveset, solve,
    diff,
    limit,
    im,
    N,
    oo
)

from sympy.calculus.util import (
    function_range,
    continuous_domain
)

from scipy.optimize import minimize, least_squares

from IPython.display import display, Markdown

import sys
import os


# Получаем абсолютный путь к директории, где находится ноутбук
notebook_dir = os.path.abspath('')
# Добавляем путь к родительской директории в sys.path
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)
    
from helper.functionanalyzer import FunctionAnalyzer


Решить головоломку. Нужно соединить  синие кружки непрерывной линией, при этом все ее отрезки - могут быть только ортогональны друг другу, а линия не должна пересекать сама себя. Т.е. через одну синюю точку искомая линия проходит только один раз. Перемещения по диагонали - нет.  Можно двигаться только через синие точки. Красные точки - пересекать нельзя. Здесь в задаче квадрат с синими и красными кружками размером 5 * 5. Соединять нужно только соседние точки. Если столбцы и строки нумеруются из левого верхнего угла (1,1) - то координаты красных точек будут: (1,1), (1,2), (2,1), (2,2), (2,3), (2,4), (3,4). Очевидно, что синяя точка (1, 3 ) имеет всего одну соседнюю  синюю точку (1,4), т.е. должна быть либо начальной, либо конечной точкой искомой линии. Учитывая, что направление линии не влияет на решение задачи - имеет смысл начать решение именно с точки (1, 3).
Для решения задачи напишем код на python, который будет перебирать и проверять все возможные варианты решения этой задачи, начиная с точки (1,3)


In [11]:
# Grid dimensions
ROWS = 5
COLS = 5

# Initialize grid: 1 for blue (traversable), 0 for red (blocked)
# Start with all blue
grid = [[1 for _ in range(COLS)] for _ in range(ROWS)]

# Mark red dots (using 0-based indexing)
# (1,1) -> grid[0][0]
# (1,2) -> grid[0][1]
# (2,1) -> grid[1][0]
# (2,2) -> grid[1][1]
# (2,3) -> grid[1][2]
# (2,4) -> grid[1][3]
# (3,4) -> grid[2][3]
red_dots_zero_based = [
    (0, 0), (0, 1),
    (1, 0), (1, 1), (1, 2), (1, 3),
    (2, 3)
]

for r, c in red_dots_zero_based:
    if 0 <= r < ROWS and 0 <= c < COLS:
        grid[r][c] = 0
    else:
        print(f"Warning: Red dot coordinate ({r+1},{c+1}) is out of bounds.")

# Calculate the total number of blue dots to visit
total_blue_dots = 0
for r in range(ROWS):
    for c in range(COLS):
        if grid[r][c] == 1:
            total_blue_dots += 1

# Define the mandatory start point (1,3) -> (0,2) in 0-based index
start_node = (0, 2)

print("Grid representation (1=Blue, 0=Red):")
for row in grid:
    print(row)
print(f"\nTotal blue dots to connect: {total_blue_dots}")
print(f"Starting search from (1-based): ({start_node[0]+1}, {start_node[1]+1})")
print("-" * 20)

import sys

# Increase recursion depth limit if needed for deep searches, though unlikely for 5x5
# sys.setrecursionlimit(2000)

def is_valid(r, c, visited):
    """Checks if a cell is within bounds, is blue, and not visited."""
    return 0 <= r < ROWS and 0 <= c < COLS and grid[r][c] == 1 and (r, c) not in visited

def solve_puzzle(start_r, start_c):
    """Initiates the DFS search for the path."""
    path = []
    visited = set()

    def dfs(r, c):
        """Recursive Depth-First Search function."""
        # Add current node to path (using 1-based for final output)
        current_node_1_based = (r + 1, c + 1)
        path.append(current_node_1_based)
        visited.add((r, c))

        # Check if we have visited all blue dots
        if len(path) == total_blue_dots:
            return path # Solution found!

        # Explore neighbors: Down, Right, Up, Left (order can affect which solution is found first)
        moves = [(1, 0), (0, 1), (-1, 0), (0, -1)]
        for dr, dc in moves:
            next_r, next_c = r + dr, c + dc
            if is_valid(next_r, next_c, visited):
                result = dfs(next_r, next_c)
                if result: # If a solution was found down this branch
                    return result # Propagate the solution back

        # Backtrack if no valid neighbor leads to a solution from here
        path.pop() # Remove current node from path
        visited.remove((r, c)) # Allow visiting this node via other paths
        return None # Indicate failure for this branch

    # Start the search from the given start node
    solution = dfs(start_r, start_c)
    return solution

# --- Run the solver ---
solution_path = solve_puzzle(start_node[0], start_node[1])

# --- Print the result ---
print("\n--- Search Result ---")
if solution_path:
    print("Solution found!")
    print("Path (sequence of 1-based coordinates):")
    # Print path clearly
    path_str = " -> ".join(map(str, solution_path))
    print(path_str)

    # Optional: Visual verification (print grid with path numbers)
    print("\nVisual Path:")
    vis_grid = [[' R ' if grid[r][c] == 0 else ' . ' for c in range(COLS)] for r in range(ROWS)]
    for i, (r, c) in enumerate(solution_path):
        vis_grid[r-1][c-1] = f"{i+1:^3}" # Center number in 3 spaces
    for row in vis_grid:
        print("".join(row))

else:
    print("No solution found starting from", (start_node[0]+1, start_node[1]+1))

Grid representation (1=Blue, 0=Red):
[0, 0, 1, 1, 1]
[0, 0, 0, 0, 1]
[1, 1, 1, 0, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]

Total blue dots to connect: 18
Starting search from (1-based): (1, 3)
--------------------

--- Search Result ---
No solution found starting from (1, 3)
